In [11]:
import cv2
import numpy as np
import pickle
import matplotlib.pyplot as plt

calibration_data = pickle.load(open('calibration_data.p','rb'))

matrix = calibration_data['camera_matrix']
dist_coef = calibration_data['distortion_coefficient']

source_points = [(580, 460), (205, 720), (1110, 720), (703, 460)]
dest_points = [(320, 0), (320, 720), (960, 720), (960, 0)]

warp_matrix = cv2.getPerspectiveTransform(np.float32(source_points), np.float32(dest_points))
inv_warp_matrix = cv2.getPerspectiveTransform(np.float32(dest_points), np.float32(source_points))

p = { 'sat_thresh': 120, 'light_thresh': 40, 'light_thresh_agr': 205,
      'grad_thresh': (0.7, 1.4), 'mag_thresh': 40, 'x_thresh': 20 }

sat_thresh = 120
light_thresh = 40
light_thresh_agr = 205
grad_min = 0.7; grad_max = 1.4
mag_thresh = 40
x_thresh = 20

In [ ]:
import cv2
import numpy as np

def birdeye_sky_view(ground_image):
    temp_image = cv2.undistort(ground_image, matrix, dist_coef, None, matrix)
    shape = (temp_image.shape[1] ,temp_image.shape[0]) # (width,height)
    # 위쪽에 이미 구해놓았음
    # warp_matrix = cv2.getPerspectiveTransform(np.float32(source_points), np.float32(dest_points))
    warp_image = cv2.warpPerspective(temp_image, warp_matrix, shape, flags=cv2.INTER_LINEAR)
    
    return warp_image

def apply_color_mask(img_hls):   
    img_l = img_hls[:, :, 1]
    img_s = img_hls[:, :, 2]
    color_cond1 = (img_s > sat_thresh) & (img_l > light_thresh)
    color_cond2 = img_l > light_thresh_agr
    b = np.zeros_like(img_s)
    b[(color_cond1 | color_cond2)] = 1
    return b

def scale_abs(x, m=255):
    x = np.absolute(x)
    x = np.uint8(m * x / np.max(x))
    return x

def roi(gray, mn = 125, mx = 1200):
  m = np.copy(gray) + 1
  m[:, :mn] = 0 
  m[:, mx:] = 0 
  return m 

def show_images(imgs, per_row = 3, per_col = 2, W = 10, H = 5, tdpi = 80):
      
  fig, ax = plt.subplots(per_col, per_row, figsize = (W, H), dpi = tdpi)
  ax = ax.ravel()
  
  for i in range(len(imgs)):
    img = imgs[i]
    ax[i].imshow(img)
  
  for i in range(per_row * per_col):
    ax[i].axis('off')

def apply_sobel_mask(img_hls):
    img_l = img_hls[:, :, 1]
    img_s = img_hls[:, :, 2]  
    img_z = np.zeros_like(img_s)     
    lx = cv2.Sobel(img_l, cv2.CV_64F, 1, 0, ksize = 5)
    ly = cv2.Sobel(img_l, cv2.CV_64F, 0, 1, ksize = 5)
    gradl = np.arctan2(np.absolute(ly), np.absolute(lx))
    l_mag = np.sqrt(lx**2 + ly**2)
    slm, slx, sly = scale_abs(l_mag), scale_abs(lx), scale_abs(ly)
    b = np.zeros_like(img_s)
    sobel_cond1 = slm > mag_thresh
    sobel_cond2 = slx > x_thresh
    sobel_cond3 = (gradl > grad_min) & (gradl < grad_max)
    b[(sobel_cond1 & sobel_cond2 & sobel_cond3)] = 1  
    return b 

def sobel_breakdown(img): #RGB
    img_hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    img_l = img_hls[:, :, 1]
    img_s = img_hls[:, :, 2]  
    img_z = np.zeros_like(img_s)     
    lx = cv2.Sobel(img_l, cv2.CV_64F, 1, 0, ksize = 5)
    ly = cv2.Sobel(img_l, cv2.CV_64F, 0, 1, ksize = 5)
    gradl = np.arctan2(np.absolute(ly), np.absolute(lx))
    l_mag = np.sqrt(lx**2 + ly**2)
    slm, slx, sly = scale_abs(l_mag), scale_abs(lx), scale_abs(ly)
    sobel_cond1 = slm > mag_thresh
    sobel_cond2 = slx > x_thresh
    sobel_cond3 = (gradl > grad_min) & (gradl < grad_max)
    b1, b2, b3 = img_z.copy(), img_z.copy(), img_z.copy()
    b1[(sobel_cond1)] = 255
    b2[(sobel_cond2)] = 255
    b3[(sobel_cond3)] = 255
    return np.dstack((b1, b2,b3))

def color_breakdown(rgb_image):
    img_hls = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2HLS)
    img_l = img_hls[:, :, 1]
    img_s = img_hls[:, :, 2]
    img_z = np.zeros_like(img_s)
    color_cond1 = (img_s > sat_thresh) & (img_l > light_thresh)
    color_cond2 = img_l > light_thresh_agr
    b1, b2 = img_z.copy(), img_z.copy()
    b1[(color_cond1)] = 255
    b2[(color_cond2)] = 255
    return np.dstack((b1, b2, img_z))

def lane_filter_apply(rgb_image):
    img_hls = cv2.cvtColor(rgb_image, cv2.COLOR_BGR2HLS)   
    color_img = apply_color_mask(img_hls)
    sobel_img = apply_sobel_mask(img_hls)
    filtered_img = cv2.bitwise_or(sobel_img, color_img)
    return filtered_img

In [ ]:
def next_y(w, h=720, window_height=80):
  y_lo = h - (w + 1) * window_height
  y_hi = h - w * window_height 
  return y_lo, y_hi

def next_x(current ,margin=100):
  x_left = current - margin
  x_right = current + margin
  return x_left, x_right

def next_midx(current, all_pixels_x, pixel_indices, min_pix=50):
    if len(pixel_indices) > min_pix:
      current = np.int(np.mean(all_pixels_x[pixel_indices]))
    return current

def indices_within_boundary(all_pixels_x, all_pixels_y, y_lo, y_hi, x_left, x_right):
  cond1 = all_pixels_y >= y_lo
  cond2 = all_pixels_y < y_hi
  cond3 = all_pixels_x >= x_left
  cond4 = all_pixels_x < x_right
  return (cond1 & cond2 & cond3 & cond4 ).nonzero()[0]


def curves_fit(binary):  
  number_of_windows = 9
  margin = 100
  minimum_pixels = 50
  ym_per_pix = 30 / 720
  xm_per_pix = 3.7 / 700
  left_pixels_indices, right_pixels_indices = [], []
  left_pixels_x, left_pixels_y = None, None
  right_pixels_x, right_pixels_y = None, None
  left_fit_curve_pix, right_fit_curve_pix = None, None
  left_fit_curve_f, right_fit_curve_f = None, None
  left_radius, right_radius = None, None
  vehicle_position, vehicle_position_words = None, None
  result = {}

  # store details
  out_img = np.dstack((binary, binary, binary)) * 255
  h, w = binary.shape[0], binary.shape[1]
  mid = h/2
  window_height = np.int(h / number_of_windows)  
  all_pixels_x = np.array(binary.nonzero()[1])
  all_pixels_y = np.array(binary.nonzero()[0]) 

  # start : w 각 세로라인별 합을 구해서 그값이 가장 큰 세로줄을 시작으로 (좌/우 구분해서)
  hist = np.sum(binary[np.int(h / 2):, :], axis = 0)
  mid = np.int(hist.shape[0] / 2)
  mid_leftx = np.argmax(hist[:mid])
  mid_rightx = np.argmax(hist[mid:]) + mid
  
  left_pixels_indices, right_pixels_indices = [], []
  x, y = [None, None, None, None], [None, None]
  
  for w in range(number_of_windows):
    y[0], y[1] = next_y(h,w)
    x[0], x[1] = next_x(mid_leftx) 
    x[2], x[3] = next_x(mid_rightx)

    cv2.rectangle(out_img, (x[0], y[0]), (x[1], y[1]), (255, 0, 0), thickness=5)  
    cv2.rectangle(out_img, (x[2], y[0]), (x[3], y[1]), (0, 255, 0), thickness=5)
    
    curr_left_pixels_indices = indices_within_boundary(all_pixels_x, all_pixels_y, y[0], y[1], x[0], x[1])
    curr_right_pixels_indices = indices_within_boundary(all_pixels_x, all_pixels_y,y[0], y[1], x[2], x[3])
    
    left_pixels_indices.append(curr_left_pixels_indices)
    right_pixels_indices.append(curr_right_pixels_indices)
    
    mid_leftx = next_midx(mid_leftx, all_pixels_x, curr_left_pixels_indices)
    mid_rightx = next_midx(mid_rightx, all_pixels_x, curr_right_pixels_indices)
  
  self.left_pixels_indices = np.concatenate(left_pixels_indices)
  self.right_pixels_indices = np.concatenate(right_pixels_indices)
  
  self.left_pixels_x, self.left_pixels_y = self.pixel_locations(self.left_pixels_indices)
  self.right_pixels_x, self.right_pixels_y = self.pixel_locations(self.right_pixels_indices)

  self.left_fit_curve_f = self.get_real_curvature(self.left_pixels_x, self.left_pixels_y)
  self.right_fit_curve_f = self.get_real_curvature(self.right_pixels_x, self.right_pixels_y)
  
  self.left_radius = self.radius_of_curvature(self.h * self.ky, self.left_fit_curve_f)
  self.right_radius = self.radius_of_curvature(self.h *  self.ky, self.right_fit_curve_f)

  self.plot()
  self.update_vehicle_position()

  result = {
    'image': self.out_img,
    'left_radius': self.left_radius,
    'right_radius': self.right_radius,
    'real_left_best_fit_curve': self.left_fit_curve_f,
    'real_right_best_fit_curve': self.right_fit_curve_f, 
    'pixel_left_best_fit_curve': self.left_fit_curve_pix,
    'pixel_right_best_fit_curve': self.right_fit_curve_pix, 
    'vehicle_position': self.vehicle_position, 
    'vehicle_position_words': self.vehicle_position_words
  }
  return result

def curve_test(path):
  img = cv2.imread(path, cv2.IMREAD_COLOR)
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  binary = lane_filter_apply(img)
  wb = np.logical_and(birdeye_sky_view(binary), roi(binary)).astype(np.uint8)
  cv2.imshow('wb',wb)
  cv2.waitKey()
  cv2.destroyAllWindows()
  result = curves_fit(wb)
  # print("[real world] left best-fit curve parameters:", result['real_left_best_fit_curve'])
  # print("[real world] right best-fit curve parameters:", result['real_right_best_fit_curve'])
  # print("[pixel] left best-fit curve parameters:", result['pixel_left_best_fit_curve'])
  # print("[pixel] left best-fit curve parameters:", result['pixel_right_best_fit_curve'])
  # print("[left] current radius of curvature:", result['left_radius'], "m")
  # print("[right] current radius of curvature:", result['right_radius'], "m")
  # print("vehicle position:", result['vehicle_position_words'])
  # plt.imshow(result['image'])

In [20]:
#curve_test('images/straight_lines1.jpg')

path = 'images/straight_lines1.jpg'
img = cv2.imread(path, cv2.IMREAD_COLOR)
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
binary = lane_filter_apply(img)
binary = np.logical_and(birdeye_sky_view(binary), roi(binary)).astype(np.uint8)
h,w = binary.shape[:2]
hist = np.sum(binary[int(h / 2):, :], axis = 0)
print(hist.shape)
mid = int(hist.shape[0] / 2)
print(mid)

(1280,)
640


In [ ]:
binary = np.array([[0,0,0,0,0,0],[0,0,0,0,0,0],[0,3,0,0,4,0],[0,1,0,0,2,0]])
print(binary)
h,w = binary.shape[:2]
print(h,w)
hist = np.sum(binary[int(h / 2):, :], axis = 0) #각 열별로 합을 구한 걸 원소로 갖는 1차원 배열. 값이 있는 곳
print(hist)
np.argmax(hist[:mid]) # 스타트지점 찾기(위 배열의 절반부분 내에서 argmax 구한거)

[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 3 0 0 4 0]
 [0 1 0 0 2 0]]
4 6
[0 4 0 0 6 0]


4

In [ ]:
arr_2d = np.array([[0, 1, 0],
                   [2, 0, 3],
                   [0, 0, 0]])

row_indices, col_indices = np.nonzero(arr_2d) #Non제로 성분들의 y 인덱스 배열, x 인덱스 배열
print(row_indices)  # 출력: [0 1 1]
print(col_indices)  # 출력: [1 0 2]  #즉 행렬의 (0,1), (1,0), (1,2)인덱스가 nonzero란 이야기. 아래 참조.

# 0이 아닌 원소에 접근
nonzero_elements_2d = arr_2d[row_indices, col_indices]
print(nonzero_elements_2d)  # 출력: [1 2 3]

# zip을 사용하여 행과 열 인덱스를 쌍으로 출력
for row, col in zip(row_indices, col_indices):
    print(f"Non-zero element found at row {row}, column {col}")

[0 1 1]
[1 0 2]
[1 2 3]
Non-zero element found at row 0, column 1
Non-zero element found at row 1, column 0
Non-zero element found at row 1, column 2
